### 1. Build the docker image for the model

In [1]:
%%sh

# The name of our algorithm
#algorithm_name=pytorch-extending-our-containers-cifar10-example
algorithm_name=model-1-graph2graph-gpu

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the SageMaker PyTorch image
$(aws ecr get-login --registry-ids 520713654638 --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --build-arg REGION=${region}
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Login Succeeded

Step 1/6 : ARG REGION=us-west-2
Step 2/6 : FROM 520713654638.dkr.ecr.$REGION.amazonaws.com/sagemaker-pytorch:0.4.0-cpu-py3
 ---> 11e92f8d88a4
Step 3/6 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 22c510b9e585
Step 4/6 : COPY /cifar10 /opt/ml/code
 ---> Using cache
 ---> e0e86498867c
Step 5/6 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Using cache
 ---> fdfa6ebaaf24
Step 6/6 : ENV SAGEMAKER_PROGRAM cifar10.py
 ---> Using cache
 ---> 913071e83d26
Successfully built 913071e83d26
Successfully tagged model-1-graph2graph-gpu:latest
The push refers to repository [739085167460.dkr.ecr.us-east-2.amazonaws.com/model-1-graph2graph-gpu]
92af458f91a3: Preparing
1e95f2314d9e: Preparing
6925e67bc96f: Preparing
92767847d831: Preparing
c6618b7eae39: Preparing
7088d75c498e: Preparing
ee905f515781: Preparing
9fb660580ee6: Preparing
ebf8767b42f1: Preparing
3db5746c911a: Preparing
819a824caf70: Preparing
647265b9d8bc: Preparing
41c002c8a6fd: Preparing
9

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

